In [1]:
setwd('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal/')
source('scripts/glmm_functions.R')

In [2]:
# Basic parameters to use.
min_cells = 10
ncpus = 10

### All memory

In [36]:
sce <- readRDS('h5ad/df.fil3_gex_bcells_vdj_sce_B_switched_mem_prog.RDS')
counts(sce) <- assays(sce)[['X']] # because i'm saving from a h5ad object with anndata2ri
sce$case_control <- factor(sce$case_control, levels = c('NEGATIVE', 'POSITIVE', 'RECOVERY'))
sce$WHO_temp_severity <- factor(sce$WHO_temp_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'))
sce$WHO_temp_severity_group <- factor(sce$WHO_temp_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'), labels = c('NA', 'mild_moderate', 'mild_moderate', 'severe_critical', 'severe_critical'))
sce$WHO_severity_group <- factor(sce$WHO_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'), labels = c('NA', 'mild_moderate', 'mild_moderate', 'severe_critical', 'severe_critical')) # interpreted as peak severity
sce$grouped_temp_severity <- ifelse(sce$WHO_temp_severity %in% c("mild", "moderate"), "mild_moderate", "severe_critical")
sce$grouped_severity <- ifelse(sce$WHO_severity %in% c("mild", "moderate"), "mild_moderate", "severe_critical")
sce$age_scaled <- scale(sce$calc_age) # scale age

In [37]:
# Remove samples with less than 10 cells
nCells <- table(sce$sample_id)
rmSamples <- names(nCells[nCells < min_cells])
sce1 <- sce[, !sce$sample_id %in% rmSamples]
# Summarize Counts
smrzd <- aggregateAcrossCells(sce1, id = as.character(colData(sce1)[, c("sample_id")]))
y <- DGEList(counts = counts(smrzd), samples = colData(smrzd))

y1 <- setupDGElist(y, "prognosis") # use grouped_temp_severity later
# sanity check
table(y1$samples$prognosis, y1$samples$individual_id)
table(y1$samples$prognosis, y1$samples$centre)
table(y1$samples$prognosis, y1$samples$sex)
table(y1$samples$prognosis, y1$samples$ethnicity)
table(y1$samples$prognosis, y1$samples$WHO_severity)
table(y1$samples$prognosis, y1$samples$WHO_temp_severity)

                
                 C20 C21 C23 C31 C33 C36 C40 C42 C58 C63 C65 C93 C123 C124 C126
  stable_disease   0   0   1   1   0   1   0   0   1   0   0   0    0    0    1
  will_improve     1   0   0   0   0   0   0   1   0   1   0   0    1    1    0
  will_worsen      0   1   0   0   1   0   1   0   0   0   1   1    0    0    0
                
                 C127 C128 C132 C137 C141 C145 C146 C147 C168 C169 C170 C190
  stable_disease    0    1    0    1    0    0    0    0    1    1    1    1
  will_improve      1    0    1    0    0    0    0    0    0    0    0    0
  will_worsen       0    0    0    0    1    1    1    1    0    0    0    0

                
                 Cambridge NCL
  stable_disease         6   5
  will_improve           0   7
  will_worsen            5   4

                
                 F M
  stable_disease 5 6
  will_improve   2 5
  will_worsen    1 8

                
                 asian black other white
  stable_disease     8     1     0     2
  will_improve       4     1     1     1
  will_worsen        4     2     0     3

                
                 critical mild moderate severe
  stable_disease        0    9        2      0
  will_improve          0    0        5      2
  will_worsen           5    0        1      3

                
                 NA mild moderate severe critical
  stable_disease  0   10        1      0        0
  will_improve    0    0        6      1        0
  will_worsen     0    6        3      0        0

In [19]:
res1 <- testDGElist(y1,
            formula = as.formula("~ prognosis + sex + ethnicity + age_scaled + centre + (1|individual_id)"),
            individual_id = "individual_id",
            modified = TRUE,
            ncores = ncpus)


n = 27 samples, 27 individuals
Time difference of 1.195185 mins

q_prognosis
-----------
Not Significant     Significant 
           2973              68 

q_sex
-----
Not Significant     Significant 
           2975              66 

q_ethnicity
-----------
Not Significant     Significant 
           2974              67 

q_age_scaled
------------
Not Significant     Significant 
           2981              60 

q_centre
--------
Not Significant     Significant 
             71            2970 


In [11]:
library(dplyr)


Attaching package: ‘dplyr’


The following object is masked from ‘package:Biobase’:

    combine


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:matrixStats’:

    count


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [20]:
print(colnames(res1$stats))

 [1] "Dispersion"            "AIC"                   "logLik"               
 [4] "(Intercept)"           "prognosiswill_improve" "prognosiswill_worsen" 
 [7] "sexM"                  "ethnicityblack"        "ethnicityother"       
[10] "ethnicitywhite"        "age_scaled"            "centreNCL"            
[13] "Chisq_prognosis"       "Chisq_sex"             "Chisq_ethnicity"      
[16] "Chisq_age_scaled"      "Chisq_centre"          "P_prognosis"          
[19] "P_sex"                 "P_ethnicity"           "P_age_scaled"         
[22] "P_centre"              "q_prognosis"           "q_sex"                
[25] "q_ethnicity"           "q_age_scaled"          "q_centre"             


In [26]:
# extract stats
extract_stats <- function(res1){
#     if (!is.na(res1)){
        anova1 <- res1$stats
        fitstats1 <- lapply(res1$fit, function(x) {
            y <- suppressWarnings(summary(x$fit))
            E <- y$coefficients[-1, 1]
            names(E) <- paste0('fit_estimates_', names(E))
            P <- y$coefficients[-1, 4]
            names(P) <- paste0('fit_P_', names(P))
            return(c(E,P))
        })
        fitstats1 <- do.call(rbind, fitstats1)
        optinfo1 <- res1$optInfo
        stats1 <- cbind(anova1, fitstats1, optinfo1)
        stats1 <- as.data.frame(stats1)
        for (i in c(
                'fit_P_prognosiswill_improve',
                'fit_P_prognosiswill_worsen',
                'fit_P_sexM',
                'fit_P_ethnicityblack',
                'fit_P_ethnicityother',
                'fit_P_ethnicitywhite',
                'fit_P_age_scaled',
                'fit_P_centreNCL')){
            stats1[,gsub('_P_', '_Q_', i)] <- p.adjust(stats1[,i], method = 'BH')
        }
        return(stats1)
    }
# }

In [27]:
results1A <- extract_stats(res1)

In [44]:
# # print(colnames(res1@stats))
# results1A <- extract_stats(res1, contrast = 'prognosis', group = 'will_improve')
results1A %>% filter(fit_Q_prognosiswill_improve < 0.05 & fit_estimates_prognosiswill_improve > 0) %>% select(c('fit_estimates_prognosiswill_improve', 'fit_Q_prognosiswill_improve'))

,fit_estimates_prognosiswill_improve,fit_Q_prognosiswill_improve
,<dbl>,<dbl>
REV3L,0.02365270,1.430037e-03
NOL8,0.09008996,4.147368e-62
MBD1,0.08487023,2.269387e-67


In [43]:
results1A %>% filter(fit_Q_prognosiswill_improve < 0.05 & fit_estimates_prognosiswill_improve < 0) %>% select(c('fit_estimates_prognosiswill_improve', 'fit_Q_prognosiswill_improve'))

,fit_estimates_prognosiswill_improve,fit_Q_prognosiswill_improve
,<dbl>,<dbl>
JUN,-0.63083238,0.000000e+00
NBPF14,-0.31254750,3.441247e-23
MCL1,-0.26872364,1.588522e-213
USF1,-0.18530412,0.000000e+00
TROVE2,-0.27053073,0.000000e+00
MDM4,-0.34745841,0.000000e+00
CAPN2,-0.26571898,3.623443e-249
ODC1,-0.32278126,0.000000e+00
SPTBN1,-0.28900852,0.000000e+00


In [42]:
results1A %>% filter(fit_Q_prognosiswill_worsen < 0.05 & fit_estimates_prognosiswill_worsen > 0) %>% select(c('fit_estimates_prognosiswill_worsen', 'fit_Q_prognosiswill_worsen'))

,fit_estimates_prognosiswill_worsen,fit_Q_prognosiswill_worsen
,<dbl>,<dbl>
JUN,0.17863119,2.872861e-44
MCL1,0.17309136,3.764491e-83
CAPN2,0.03848899,4.577042e-05
TUBA4A,0.09406721,9.980793e-57
APEH,0.47332712,0.000000e+00
PJA2,0.09407091,3.071256e-69
ATF6B,0.05621016,4.275608e-29
FGD2,0.36845221,0.000000e+00
ZBTB24,0.07730751,4.563285e-32


In [39]:
results1A %>% filter(fit_Q_prognosiswill_worsen < 0.05 & fit_estimates_prognosiswill_worsen < 0)

,Dispersion,AIC,logLik,(Intercept),prognosiswill_improve,prognosiswill_worsen,sexM,ethnicityblack,ethnicityother,ethnicitywhite,⋯,Singular,Conv,fit_Q_prognosiswill_improve,fit_Q_prognosiswill_worsen,fit_Q_sexM,fit_Q_ethnicityblack,fit_Q_ethnicityother,fit_Q_ethnicitywhite,fit_Q_age_scaled,fit_Q_centreNCL
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
USF1,0.11779129,193.6740,-85.83702,1.694081,-0.18530412,-0.19288005,-0.75700944,-1.39982306,-1.4539809,-0.323315395,⋯,0,1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.8187132,0.000000e+00,0.000000e+00,0
TROVE2,0.11504949,200.9980,-89.49898,1.642816,-0.27053073,-0.13923292,-0.55564069,-0.63289640,-0.9356403,-0.027686131,⋯,0,1,0.000000e+00,8.371337e-203,0.000000e+00,0.000000e+00,0.8187132,6.008047e-08,0.000000e+00,0
MDM4,0.11770845,231.9543,-104.97716,2.468519,-0.34745841,-0.16678113,-0.17755846,-0.47355298,-1.7297246,-0.293151325,⋯,0,1,0.000000e+00,3.854840e-158,2.692158e-179,0.000000e+00,0.8187132,0.000000e+00,7.187011e-17,0
ODC1,0.03436864,270.1860,-124.09302,2.884232,-0.32278126,-0.46367358,-0.22228442,-0.54163516,-0.5318500,-0.129241237,⋯,0,1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.8187132,2.626878e-137,9.387076e-77,0
SPTBN1,0.12401837,230.2615,-104.13073,2.509553,-0.28900852,-0.35513599,-0.25793378,-0.93010903,-1.1753711,-0.269119004,⋯,0,1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.8187132,0.000000e+00,0.000000e+00,0
AFTPH,0.02021463,234.3331,-106.16656,2.357462,-0.26187755,-0.27033818,-0.31087005,-0.83989545,-1.4448276,-0.349216441,⋯,0,1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.8187132,0.000000e+00,6.177619e-111,0
TTN,0.12084266,232.1581,-105.07905,2.597024,-0.46068113,-0.57594774,-0.05527889,-0.90034404,-0.8387631,-0.386689390,⋯,0,1,0.000000e+00,0.000000e+00,4.226635e-16,0.000000e+00,0.8187132,0.000000e+00,4.318323e-149,0
TMBIM1,0.13302223,213.9019,-95.95095,1.519649,-0.42509562,-0.09391063,-0.23020992,-1.64898783,-0.8532816,-0.784811034,⋯,0,1,0.000000e+00,5.680111e-101,0.000000e+00,0.000000e+00,0.8187132,0.000000e+00,0.000000e+00,0
PRKCD,0.11692852,215.7225,-96.86127,1.863177,-0.24902702,-0.08806931,-0.52361966,-1.12616568,-1.3064276,-0.087584546,⋯,0,1,0.000000e+00,2.195653e-70,0.000000e+00,0.000000e+00,0.8187132,1.062673e-69,0.000000e+00,0


In [35]:
res2 <- testDGElist(y1,
            formula = as.formula("~ prognosis*WHO_severity + sex + ethnicity + age_scaled + centre + (1|individual_id)"),
            individual_id = "individual_id",
            ncores = ncpus)


n = 27 samples, 27 individuals
Time difference of 2.778521 mins


ERROR: Error in glmmSeq(formula, id = individual_id, countdata = dgelist$counts, : All genes returned an error. Check sufficient data in each group
